## What is NAIP?
In this chapter, you will work with NAIP data.

>The National Agriculture Imagery Program (NAIP) acquires aerial imagery during the agricultural growing seasons in the continental U.S. A primary goal of the NAIP program is to make digital ortho photography available to governmental agencies and the public within a year of acquisition.

>NAIP is administered by the USDA’s Farm Service Agency (FSA) through the Aerial Photography Field Office in Salt Lake City. This “leaf-on” imagery is used as a base layer for GIS programs in FSA’s County Service Centers, and is used to maintain the Common Land Unit (CLU) boundaries. – USDA NAIP Program

[Read more about NAIP](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/intro-naip/#:~:text=Read%20more%20about%20NAIP)

NAIP is a great source of high resolution imagery across the United States. NAIP imagery is often collected with just a red, green and Blue band. However, some flights include a near infrared band which is very useful for quantifying vegetation cover and health.

NAIP data access: The data used in this lesson were downloaded from the USGS Earth explorer website.

## Open NAIP Data in Python
Next, you will use NAIP imagery for the Coldsprings fire study area in Colorado. To work with multi-band raster data you will use the rioxarray and geopandas packages. You will also use the plot module from the earthpy package for raster plotting.

Before you get started, make sure that your working directory is set.

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import rioxarray as rxr
import geopandas as gpd




plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['axes.titlesize'] = 20

To begin, you will use the rioxarray open_rasterio function to open the multi-band NAIP image

```rxr.open_rasterio("path-to-tif-file-here")```

Don’t forget that with rioxarray you can automatically mask out the fill values of a raster with the argument ```masked=True``` in ```open_rasterio```.

In [ ]:
naip_csf_path = os.path.join("data","cold-springs-fire", 
                             "naip", 
                             "m_3910505_nw_13_1_20150919", 
                             "crop", 
                             "m_3910505_nw_13_1_20150919_crop.tif")

naip_csf = rxr.open_rasterio(naip_csf_path, masked=True)
naip_csf

Above you imported a geotiff like you’ve done before. But this file is different. Notice the shape of the resulting numpy array. How many layers (known as bands) does it have?


In [ ]:
naip_csf.shape


Just like you’ve done before, you can plot a single band in the NAIP raster using ```imshow()```. However, now that you have multiple layers or bands, you need to tell ```imshow()``` what layer you wish to plot. Use ```arrayname[0]``` to plot the first band of the image.

In [ ]:
fig, ax = plt.subplots()

ax.imshow(naip_csf[0], 
          cmap="Greys_r")
ax.set_title("NAIP RGB Imagery Band 1 Red \nCold Springs Fire Scar")
ax.set_axis_off()
plt.show()

Look closely at the .band element of your raster. Note that now, there are four bands instead of one. This is because you have multiple bands in your raster, one for each ‘color’ or type of light collected by the camera. For NAIP data you have red, green, blue and near infrared bands. When you worked with the lidar rasters in week 2 your count was 1 as a DSM or DTM is only composed of one band.



In [ ]:
naip_csf.band

### Image Raster Data Values
Next, examine the raster’s min and max values. What is the value range?

In [ ]:
# View min and max value
print(naip_csf.min())
print(naip_csf.max())

This raster contains values between 0 and 255. These values represent degrees of brightness associated with the image band. In the case of a RGB image (red, green and blue), band 1 is the red band. When we plot the red band, larger numbers (towards 255) represent pixels with more red in them (a strong red reflection). Smaller numbers (towards 0) represent pixels with less red in them (less red was reflected).

To plot an RGB image, we mix red + green + blue values, using the ratio of each. The ratio of each color is determined by how much light was recorded (the reflectance value) in each band. This mixture creates one single color that, in turn, makes up the full color image - similar to the color image that your camera phone creates.

## 8 vs 16 Bit Images
It’s important to note that this image is an 8 bit image. This means that all values in the raster are stored within a range of 0:255. This differs from a 16-bit image, in which values can be stored within a range of 0:65,535.

In these lessons, you will work with 8-bit images. For 8-bit images, the brightest whites will be at or close to 255. The darkest values in each band will be closer to 0.

### Plot A Specific Band
You can plot a single band of your choice using numpy indexing. ```naip_csf[1]``` will access just the second band - which is the green band when using NAIP data.

In [ ]:
fig, ax = plt.subplots()

ax.imshow(naip_csf[1], 
          cmap="Greys_r")
ax.set_title("RGB Imagery - Band 2 - Green\nCold Springs Fire Scar")
ax.set_axis_off()
plt.show()

## Rasters and Numpy Arrays - A Review
Remember that when you import a raster dataset into Python, the data are converted to an **xarray** object. A numpy array has no inherent spatial information attached to it, nor does an **xarray** object. The data are just a matrix of values. This makes processing the data fast.

The spatial information for the raster is stored in a ```.rio``` attribute which is available if you import rioxarray in your workflow. This rio attribute allows you to export the data as a geotiff or other spatial format.

### Plot Raster Band Images
Next plot each band in the raster. This is another intermediate step (like plotting histograms) that you might want to do when you first explore and open your data. You will not need this for your homework but you might want to do it to explore other data that you use in your career. 

Similar to plotting a single band, in each band “color”, the brightest pixels are lighter in color or white representing a stronger reflectance for that color on that pixel. The darkest pixels are darker to black in color representing less reflectance of that color in that pixel.


In [ ]:
titles = ["Red Band", "Green Band", "Blue Band", "Near Infrared (NIR) Band"]
plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=0.5)
plt.suptitle("NAIP Bands", fontsize=18, y=0.95)

for n, bands in enumerate(naip_csf):
    # add a new subplot iteratively
    ax = plt.subplot(2, 2, n + 1)

    # filter df and plot ticker on the new subplot axis

    ax.imshow(naip_csf[n], 
          cmap="Greys_r")

    # chart formatting
    ax.set_title(titles[n])
    #ax.get_legend().remove()
    ax.set_xlabel("")

## Plot RGB Data in Python
Previously you have plotted individual bands using a greyscale color ramp in Python. Next, you will learn how to plot an RGB composite image. This type of image is similar in appearance to one you capture using a cell phone or digital camera.

![_._](img/RGB-bands-raster-stack.jpg)


A "true" color image consists of 3 bands - red, green and blue. When composited or rendered together in a GIS, or even a image-editor like Photoshop the bands create a color image. Source: Colin Williams, NEON.
You can use create a temporary rioxarray dataframe with just the 3 bands you're interested in plotting. Let's start by isolating the RGB bands of the dataset:

In [ ]:
rgb_temp = (naip_csf[0:3,:,:])
rgb_temp.shape

While other plot methods require the DataArray to be strictly two-dimensional, imshow also accepts a 3D array where some dimension can be interpreted as RGB or RGBA color channels and allows this dimension to be specified via the kwarg ```rgb=```.

In [ ]:
rgb_temp.astype("int").plot.imshow(rgb="band",figsize=(10, 8))

### Image Stretch To Increase Contrast
The image above looks pretty good. You can explore whether applying a stretch to the image improves clarity and contrast.

![_._](img/raster-image-stretch-dark.jpg)

When the range of pixel brightness values is closer to 0, a darker image is rendered by default. You can stretch the values to extend to the full 0-255 range of potential values to increase the visual contrast of the image.

![_._](img/raster-image-stretch-light.jpg)

When the range of pixel brightness values is closer to 255, a lighter image is rendered by default. You can stretch the values to extend to the full 0-255 range of potential values to increase the visual contrast of the image.



Unlike ```matplotlib.pyplot.imshow()```, which ignores vmin/vmax for RGB(A) data, xarray will use vmin and vmax for RGB(A) data by applying a single scaling factor and offset to all bands. Passing robust=True infers vmin and vmax in the usual way. Additionally the y-axis is not inverted by default, you can restore the matplotlib behavior by setting yincrease=False.


In [ ]:
rgb_temp.astype("int").plot.imshow(rgb="band",figsize=(10, 8),robust = True)

What does the image look like using a different stretch? Any better? worse?

In this case, the stretch does increase the contrast in our image. However visually it may or may not be what you want to plot.



You also create a false color image that might show additional spatial information more clearly. Here we use the NIR, red and green bands to create a color infrared (CIR) plot:

In [ ]:
cir_temp = (naip_csf[[3,0,1],:,:])
cir_temp.shape

In [ ]:
cir_temp.astype("int").plot.imshow(rgb="band",robust = True,figsize=(10, 8))
